In [ ]:
# -------------------------------------------------------------------------
# MODIFY WITH CARE
# Standard libraries to be used in AWS Glue jobs
# -------------------------------------------------------------------------

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import functions as f
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, array, ArrayType, DateType
from pyspark.sql import Row, Column
import datetime
import json
import boto3
import time
import logging
import calendar
import uuid
from dateutil import relativedelta
from dateutil.relativedelta import relativedelta
from awsglue.utils import getResolvedOptions

In [ ]:
# Edit Variables below as per AWS account

BATCH_MODE=False

if BATCH_MODE:
    args = getResolvedOptions(sys.argv, ['JOB_NAME','ARG_JOB_DATE'])
    JOB_DATE=args['ARG_JOB_DATE']
else:
    JOB_DATE='20200708' 

S3PATHREAD="s3://data-bucket-1234/data/cricket-world-cup/"+JOB_DATE+"/"
S3PATHWRITE="s3://data-bucket-1234/curated/cricket-world-cup"

PARTITION='dt='+JOB_DATE
CATALOG_TABLE_LIST=[]

CURATED_DATABASE='curated_worldcup'

BOWLER_TBL='bowlers'

CRAWLER_NAME='worldcup'
CRAWLER_ARN='arn:aws:iam::175908995626:role/glue-role'

In [ ]:
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
client = boto3.client('glue', region_name='us-east-1')

In [ ]:
def read_s3_file(spark, type, path, delimiter='|', header='true'):
    if (type == 'CSV'):
        return spark.read.format("com.databricks.spark.csv").option("header", header).option("delimiter", delimiter).load(path)
    if (type == 'PARQUET'):
        return spark.read.parquet(path)
    
def write_s3_file(df, table_location, table, partition=None, format='PARQUET', delimiter='\t', coalesce=1, header=False):
    if format == 'PARQUET':
        df.write.parquet(table_location+'/'+table+'/'+partition, mode = "overwrite")
    if format == 'CSV':
        df.coalesce(coalesce).write.option("delimiter", delimiter).option("quote", "\"").option("quoteAll", "true").csv(table_location +'/' + partition)
        
def append_path_to_list(list, location, table_name):
    list.append({'Path': location + '/' + table_name})
    
def delete_catalog_table(client, database, table):
    try:
        response = client.delete_table(DatabaseName=database,Name=table)
    except Exception as e:
        print(table+' does not exist in glue catalog')
        
def create_crawler(client, crawler_name, iam_role_arn, database_name):
    return client.create_crawler(
        Name=crawler_name,
        Role=iam_role_arn,
        DatabaseName=database_name,
        Targets={
            'S3Targets':[
                {'Path':'s3://bucket/placeholder'}
            ]}
        )
    
def update_crawler(client, crawler_name, s3targets):
    client.update_crawler(
        Name=crawler_name,
        Targets = {'S3Targets':s3targets}
            
    )
    
def start_crawler(client, crawler_name):
    print(crawler_name + ' started.')
    
    # Getting PRE-RUN READY status.
    while(True):
        time.sleep(1)
        response = client.get_crawler(
                        Name=crawler_name
                   )
        
        if response['Crawler']['State'] == 'READY':
            print(response['Crawler']['State'])
            break
            
    client.start_crawler(
        Name=crawler_name
    )
    
    # Getting RUNNING status for stdout.            
    while(True):
        time.sleep(15)
        response = client.get_crawler(
                        Name=crawler_name
                   )
        
        if response['Crawler']['State'] == 'RUNNING':
            print(response['Crawler']['State'])
            break
        
    # Getting STOPPING status for stdout.
    while(True):
        time.sleep(1)
        response = client.get_crawler(
                        Name=crawler_name
                   )
        
        if response['Crawler']['State'] == 'STOPPING':
            print(response['Crawler']['State'])
            break
    
   # Getting READY status.
    while(True):
        time.sleep(1)
        response = client.get_crawler(
                        Name=crawler_name
                   )
        
        if response['Crawler']['State'] == 'READY':
            print(response['Crawler']['State'])
            break

def delete_crawler(client, crawler_name):
    # Getting READY status before deleting making sure it won't delete a running crawler.
    while(True):
        time.sleep(1)
        response = client.get_crawler(
                        Name=crawler_name
                   )
        
        if response['Crawler']['State'] == 'READY':
            print(response['Crawler']['State'])
            break
    
    client.delete_crawler(
        Name=crawler_name
    )
    
    print(crawler_name + ' deleted.')

In [ ]:
bowler_df=read_s3_file(spark, 'CSV', S3PATHREAD+"Bowler_data.csv", delimiter=',',header='true')
bowler_df.show(5)

In [ ]:
bowler_df=bowler_df \
          .withColumnRenamed('_c0','ID') \
          .withColumn('Overs', f.col('Overs').cast(IntegerType())) \
          .withColumn('Mdns', f.col('Mdns').cast(IntegerType())) \
          .withColumn('Runs', f.col('Runs').cast(IntegerType())) \
          .withColumn('Wkts', f.col('Wkts').cast(IntegerType())) \
          .withColumn('Econ', f.col('Econ').cast(IntegerType())) \
          .withColumn('Ave', f.col('Ave').cast(IntegerType())) \
          .withColumn('Player_ID', f.col('Player_ID').cast(IntegerType()))

bowler_df.show(5)

In [ ]:
bowler_df.printSchema()

In [ ]:
bowler_df.registerTempTable('bowlers')

In [ ]:
bowler_df = spark.sql(" SELECT `ID`,`Overs`,`Mdns`,`Runs`,`Wkts`,`Econ`,`Ave`,`SR`, " \
                      " SUBSTR(`Opposition`, 2,20) as `Opposition`, " \
                      " `Ground`, TO_DATE(`Start Date`, 'dd MMM yyyy') AS Start_Date, " \
                      " SUBSTR(Match_ID, 6,10) AS Match_Number,`Bowler`,`Player_ID` " \
                      " FROM bowlers")
bowler_df.count()

In [ ]:
write_s3_file(bowler_df, S3PATHWRITE, BOWLER_TBL, PARTITION)
append_path_to_list(CATALOG_TABLE_LIST, S3PATHWRITE, BOWLER_TBL) 
print(CATALOG_TABLE_LIST)

In [ ]:
# Filtering using Dataframe commands
bowler_df.filter(bowler_df.Player_ID == 49619).count()

In [ ]:
# Filtering using Spark SQL commands
sqlContext.sql("SELECT * FROM bowlers where Player_ID = 49619").count()

In [ ]:
# Average overs did a bowler bowl
avg_bowler_df = bowler_df.groupBy(bowler_df.Player_ID).avg('Overs')

In [ ]:
avg_bowler_df.show()

In [ ]:
avg_bowler_df = spark.sql("SELECT Player_ID, avg(Overs) FROM bowlers GROUP BY Player_ID")

In [ ]:
avg_bowler_df.show()

In [ ]:
# Bowlers with lowest Economy Rate per over
econ_bowler_df = spark.sql("SELECT Player_ID, (SELECT min(Econ) FROM bowlers) as Econ " \
                            " FROM bowlers WHERE Econ=(SELECT min(Econ) FROM bowlers)")
econ_bowler_df.show()

In [ ]:
# Match Numbers played in Port of Spain
econ_bowler_df = spark.sql("SELECT DISTINCT SUBSTR(Match_ID, 6,10) AS Match_Number " \
                            " FROM bowlers WHERE Ground='Port of Spain'")
econ_bowler_df.show()

In [ ]:
# -------------------------------------------------------------------------
# Script to delete previously catalogued tables
# -------------------------------------------------------------------------

delete_catalog_table(client, CURATED_DATABASE, BOWLER_TBL) 

In [ ]:
# -------------------------------------------------------------------------
# Crawl tables
# -------------------------------------------------------------------------
create_crawler(client, CRAWLER_NAME, CRAWLER_ARN, CURATED_DATABASE)
update_crawler(client, CRAWLER_NAME, CATALOG_TABLE_LIST)
start_crawler(client,  CRAWLER_NAME)
delete_crawler(client, CRAWLER_NAME)